In [13]:
import numpy as np
import cv2
from skimage import io
from PIL import Image
from IPython.core.display import display, HTML
from skimage.transform import resize
from skimage import io
from os import listdir
from os.path import isfile, join
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn import datasets, svm, metrics
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import roc_auc_score

display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline

In [14]:
# load models 
face_cascade = cv2.CascadeClassifier("/home/masta/Downloads/opencv-2.4.11/data/haarcascades/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("/home/masta/Downloads/opencv-2.4.11/data/haarcascades/haarcascade_lefteye_2splits.xml")
recognizer = cv2.createLBPHFaceRecognizer()

# classificator
with open('/home/masta/Downloads/Dataset_A_Eye_Images/closed_open_eyes.pkl', 'rb') as fid:
    clf2 = cPickle.load(fid)
# scaller
with open('/home/masta/Downloads/Dataset_A_Eye_Images/StSc.pkl', 'rb') as sc:
    StSc = cPickle.load(sc)

In [18]:
# prevent some cv2 artifacts
def eyes_selection(eyes):
    eyes_checked = []
    for (ex1,ey1,ew1,eh1) in eyes:
        for (ex2,ey2,ew2,eh2) in eyes:
            if (0.7 < (float((ew1)*(eh1))/((ew2)*(eh2))) < 1.3) and\
            (0.7 < float(ew1)/(ew2) < 1.3) and\
            (0.7 < float(eh1)/(eh2) < 1.3) and\
            (ex1 != ex2) and\
            (min(ey1+eh1,ey2+eh2)-max(ey1, ey2) > min(eh1,eh2)/2.5) and\
            ([ex1,ey1,ew1,eh1] not in eyes_checked):
                eyes_checked.append([ex1,ey1,ew1,eh1])
    return eyes_checked

# main function 
def closed_open_eyes_detection(img):
    token = 0
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        eyes = eyes_selection(eyes)
        for (ex,ey,ew,eh) in eyes:
            if clf.predict(StSc.transform(resize(roi_gray[ey:ey+eh, ex:ex+ew], (28, 28)).reshape(784))) != 1:
                token += 1
    if token > 0: 
        return 'closed'
    else: 
        return 'open'

In [19]:
# Demo
import warnings
warnings.filterwarnings('ignore')
cap = cv2.VideoCapture(0)

i = 0

while i < 100:
    i += 1
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        eyes = eyes_selection(eyes)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            if clf.predict(StSc.transform(resize(roi_gray[ey:ey+eh, ex:ex+ew], (28, 28)).reshape(784))) == 1:
                cv2.putText(roi_color,'Open',(ex,ey), 1,1, (255,255,255))
            else:
                cv2.putText(roi_color,'Closed',(ex,ey), 1,1, (255,255,255))

    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [2]:
#Train | Test data loading

from skimage import io
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
path = '/home/masta/Downloads/Dataset_A_Eye_Images/new_closed/'
  
closed_eyes_train = [f for f in listdir(path) if isfile(join(path, f))]
closed_eyes_data = list([])
for i in closed_eyes_train:
    img = io.imread(path + i)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex1,ey1,ew1,eh1) in eyes:
            if len((ex1,ey1,ew1,eh1))>0:
                closed_eyes_data.append(resize(roi_gray[ey1:ey1+eh1, ex1:ex1+ew1], (28, 28)).reshape(784))
                closed_eyes_data.append(resize(cv2.flip(roi_gray[ey1:ey1+eh1, ex1:ex1+ew1],1), (28, 28)).reshape(784))

closed_target = np.empty(len(closed_eyes_data))
closed_target.fill(0)

path = '/home/masta/Downloads/Dataset_A_Eye_Images/new_open/'
open_eyes_train = [f for f in listdir(path) if isfile(join(path, f))]
open_eyes_data = list([])
for i in open_eyes_train:
    img = io.imread(path + i)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex1,ey1,ew1,eh1) in eyes:
            if len((ex1,ey1,ew1,eh1))>0:
                open_eyes_data.append(resize(roi_gray[ey1:ey1+eh1, ex1:ex1+ew1], (28, 28)).reshape(784))


open_target = np.empty(len(open_eyes_data))
open_target.fill(1)

full_data = np.append(closed_eyes_data,open_eyes_data, axis = 0)
target = np.append(closed_target,open_target, axis = 0)

In [3]:
# SVM learning
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn import datasets, svm, metrics
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import roc_auc_score

X = full_data
StSc = StandardScaler().fit(X)
X = StSc.transform(X)
y = target
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=42)

clf = svm.SVC(kernel='linear', gamma=0.01)

clf.fit(X_train, y_train)
scores = cross_validation.cross_val_score(clf, X, y, cv=5)

print ("ROC AUC: %0.2f" % roc_auc_score(y_test, clf.predict(X_test)))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

ROC AUC: 0.94
Accuracy: 0.94 (+/- 0.03)


In [11]:
import cPickle
# save the classifier:
with open('/home/masta/Downloads/Dataset_A_Eye_Images/closed_open_eyes.pkl', 'wb') as fid:
    cPickle.dump(clf, fid)
    
# save the scaller:
with open('/home/masta/Downloads/Dataset_A_Eye_Images/StSc.pkl', 'wb') as fid:
    cPickle.dump(StSc, fid)